# Make geojson for pulling MODIS data
- Read the site metadata generated in the previous step
- Calculate a 4km x 4km bounding box in WGS84
- Ensure time format is compatible with colab script

In [1]:
import pandas as pd
from shapely.geometry import Polygon
from pyproj import Proj, transform, CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import geopandas as gpd
from pathlib import Path

/tmp/ipykernel_6908/2581614532.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
META_DIR = Path('data/meta')
meta = pd.read_csv(META_DIR / 'processed_site_meta.csv')
meta

,SITE_ID,LOCATION_LAT,LOCATION_LON,LOCATION_ELEV,IGBP,TIME_INFO
0,AR-SLu,-33.464800,-66.459800,NaN,MF,"{'fluxnet': ['2009_01_01', '2011_12_31']}"
1,AR-Vir,-28.239500,-56.188600,NaN,ENF,"{'fluxnet': ['2009_01_01', '2012_12_31']}"
2,AT-Neu,47.116700,11.317500,970.0,GRA,"{'fluxnet': ['2002_01_01', '2012_12_31']}"
3,AU-ASM,-22.283000,133.249000,NaN,SAV,"{'fluxnet': ['2010_01_01', '2014_12_31']}"
4,AU-Ade,-13.076900,131.117800,NaN,WSA,"{'fluxnet': ['2007_01_01', '2009_12_31']}"
...,...,...,...,...,...,...
381,FI-Qvd,60.295242,22.391607,NaN,CRO,"{'icos-ww': ['2018_01_01', '2020_12_31']}"
382,FR-FBn,43.240790,5.678650,436.0,ENF,"{'icos-ww': ['2008_01_01', '2020_12_31']}"
383,IE-Cra,53.323090,-7.641774,NaN,WET,"{'icos-ww': ['2020_01_01', '2020_12_31']}"
384,IL-Yat,31.345045,35.051989,NaN,ENF,"{'icos-ww': ['2000_01_01', '2020_12_31']}"


In [3]:
def get_4km_polygon_wkt(lat, lon):
    utm_crs_info = query_utm_crs_info(
        area_of_interest=AreaOfInterest(west_lon_degree=lon, south_lat_degree=lat, east_lon_degree=lon, north_lat_degree=lat),
        datum_name="WGS 84"
    )[0]
    utm_crs = CRS.from_epsg(utm_crs_info.code)

    # Projected coordinate system for accurate distance measurement
    proj_utm = Proj(utm_crs)

    # Convert the given lat/lon in WGS84 to UTM coordinates
    x, y = proj_utm(lon, lat)

    # Define the offsets (2km in each direction to form a 4km box)
    offset = 2000  # meters

    # Calculate the corners of the box in UTM coordinates
    bottom_left = (x - offset, y - offset)
    bottom_right = (x + offset, y - offset)
    top_right = (x + offset, y + offset)
    top_left = (x - offset, y + offset)

    # Create a polygon from these corners
    box = Polygon([bottom_left, bottom_right, top_right, top_left, bottom_left])

    # Optionally, convert the polygon back to geographic coordinates (WGS84)
    proj_wgs84 = Proj(proj='latlong', datum='WGS84')
    lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)

    # Create a new polygon in geographic coordinates
    geo_polygon = Polygon(zip(lon_lat_polygon[0], lon_lat_polygon[1]))

    return geo_polygon

meta['geometry'] = meta.apply(lambda row: get_4km_polygon_wkt(row['LOCATION_LAT'], row['LOCATION_LON']), axis=1)
meta

/tmp/ipykernel_6908/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_6908/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_6908/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_6908/754094957.py:28: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon_lat_polygon = transform(proj_utm, proj_wgs84, *box.exterior.xy)
/tmp/ipykernel_6908/

,SITE_ID,LOCATION_LAT,LOCATION_LON,LOCATION_ELEV,IGBP,TIME_INFO,geometry
0,AR-SLu,-33.464800,-66.459800,NaN,MF,"{'fluxnet': ['2009_01_01', '2011_12_31']}",POLYGON ((-66.48078041762312 -33.4832609014259...
1,AR-Vir,-28.239500,-56.188600,NaN,ENF,"{'fluxnet': ['2009_01_01', '2012_12_31']}","POLYGON ((-56.20885125145949 -28.257671662865,..."
2,AT-Neu,47.116700,11.317500,970.0,GRA,"{'fluxnet': ['2002_01_01', '2012_12_31']}","POLYGON ((11.29038488183552 47.09924744393096,..."
3,AU-ASM,-22.283000,133.249000,NaN,SAV,"{'fluxnet': ['2010_01_01', '2014_12_31']}",POLYGON ((133.22936698745332 -22.3008493153538...
4,AU-Ade,-13.076900,131.117800,NaN,WSA,"{'fluxnet': ['2007_01_01', '2009_12_31']}",POLYGON ((131.09951674779168 -13.0951236588483...
...,...,...,...,...,...,...,...
381,FI-Qvd,60.295242,22.391607,NaN,CRO,"{'icos-ww': ['2018_01_01', '2020_12_31']}",POLYGON ((22.354694818780292 60.27766316768692...
382,FR-FBn,43.240790,5.678650,436.0,ENF,"{'icos-ww': ['2008_01_01', '2020_12_31']}",POLYGON ((5.653261737940925 43.223373849759085...
383,IE-Cra,53.323090,-7.641774,NaN,WET,"{'icos-ww': ['2020_01_01', '2020_12_31']}",POLYGON ((-7.672350046581642 53.30545491896451...
384,IL-Yat,31.345045,35.051989,NaN,ENF,"{'icos-ww': ['2000_01_01', '2020_12_31']}","POLYGON ((35.03058847187028 31.32734485249443,..."


In [4]:
gdf = gpd.GeoDataFrame(meta, crs='EPSG:4326', geometry=meta['geometry'])
gdf.to_file(META_DIR / 'sites.geojson', driver='GeoJSON')